<a href="https://colab.research.google.com/github/fani-lab/Adila/blob/main/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

**Setup Python 3.8 in Colab**
- No need to create an environment as we are already in an environment/container


---



In [6]:
!sudo apt-get update -y
!sudo apt-get install -y python3.8 python3.8-venv python3.8-distutils python3-pip
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 10
!python --version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,901 kB]
Fetched 5,758 kB in 3s (1,663 k

In [8]:
!rm -R adila/
!git clone https://github.com/Fani-Lab/adila
!pip install --upgrade pip setuptools
!pip install -r adila/requirements.txt

rm: cannot remove 'adila/': No such file or directory
Cloning into 'adila'...
remote: Enumerating objects: 1674, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 1674 (delta 138), reused 128 (delta 81), pack-reused 1442 (from 2)
Receiving objects: 100% (1674/1674), 17.47 MiB | 15.16 MiB/s, done.
Resolving deltas: 100% (891/891), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.0/806.0 kB 28.1 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144552 sha256=bb0cbf1b38126ddb01b0d1ec225d8921e7f9d221c3d37f9c7ec9c054d12efa35
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime
  Attempting uninstall: zipp
    Found existing installation



---


**1. Running Adila (Single)**

[`Hydra`](https://hydra.cc/) config overriding `data.*`, `fair.*`, and `eval.*` settings in [`Adila/src/__config__.yaml`](https://github.com/fani-lab/Adila/blob/main/src/__config__.yaml)



---



  - `data.fteamsvecs`, the sparse matrix representation of the entire dataset (teams), including `['member']` and `['skill']`, each row showing a `team`
  - `data.fsplits`, the train-test split rowids of `data.fteamsvecs`
  - `data.fpred`, contains the member recommendations for each team in the test split, of size `|test|×|member|`. The split file is necessary to know these recommendations are for what teams in the dataset
  
  These files can be obtained from by runnig `OpeNTF` team recommendation library at [`OpeNTF`](https://github.com/fani-lab/OpeNTF) like for the `toy.dblp` dataset at [`OpeNTF/data/dblp/toy.dblp.v12.json`](https://github.com/fani-lab/OpeNTF/tree/main/data/dblp/toy.dblp.v12.json).
  
  Adila codebase already included them at [`Adila/output/dblp/toy.dblp.v12.json`](https://github.com/fani-lab/Adila/tree/main/output/dblp/toy.dblp.v12.json)
  - `data.fgender`, contains the `female` member ids as the `minority` also `protected` group.
  
  In `Adila`, we keep the labels of `minority` group for efficiency as they are very few relative to majority group. However, `protected` group could be the same as `minority` group, like in `gender` protected attribute, or the `majority` group (non-popular experts, who are more often than popular ones), like in `popularity` protected attribute.

  - `data.output`, which is the path of generated files. As seen in the codebase, it is suggested to follow same folder structure as in [`OpeNTF`](https://github.com/fani-lab/OpeNTF) to avoid conflicts between prediction files of different baselines.



```
data:
  fteamsvecs: ../output/dblp/toy.dblp.v12.json/teamsvecs.pkl
  fgender: ../output/dblp/toy.dblp.v12.json/females.csv
  fsplits: ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85.pkl
  fpred: ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/f0.test.pred
  output: ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000
```

---

  - `fair.algorithm`, reranking algorithm from `det_greedy`, `det_cons`, `det_relaxed`, `det_const_sort`, `fa-ir`
  - `notion`, notion of fairness, from equality of odds (`eo`), demographic parity (`dp`)
  - `k_max`, cutoff for the reranking algorithms
  - `attribute`, the protected attribute from `popularity`, `gender`
  - `is_popular_alg`, popularity status based on either `avg` teams per experts, i.e., whoever above this value is popular, or `auc` whoever is in the head of distribution figure
  - `alpha`, the significance value for fa-ir algorithm


```
fair:
  algorithm: det_greedy
  notion: eo
  k_max: 5
  attribute: popularity
  is_popular_alg: avg
  alpha: 0.1
```
---

  - `per_instance`, metric value for each test instance of team, needed for paired significance tests
  - `fair_metrics`, metrics to measure fairness of the recommended members for the test teams
  - `utility_metrics`, metrics to measure accuracy of the recommended members for the test teams
  - `topk`, cutoff for the top-k recommended members
  - `trec`, ranking metrics from `trec_eval`. See the complete list at [`pytrec_eval`](https://github.com/terrierteam/pytrec_eval/blob/master/tests/pytrec_eval_tests.py)

```
eval:
  per_instance: True
  fair_metrics: [ndkl, skew]
  utility_metrics:
    topk: '2,5'
    trec: [P_topk, recall_topk, ndcg_cut_topk, map_cut_topk, success_topk]
```




In [9]:
%cd adila/src/
!python main.py data.fpred=../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/f0.test.pred fair.algorithm=fa-ir fair.notion=eo fair.attribute=popularity fair.is_popular_alg=avg

/content/adila/src
[2025-12-09 05:43:33,776][__main__][INFO] - Loading stats, ratios, and ids for minority experts ...
main.py:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature

**Output Folders and Files**

- `adila/{popularity,gender}/stats.pkl`, the distribution of popular/non-popular, or male/female experts in the entire dataset
- `adila/{popularity,gender}/labels.csv`, the ids for popular or female experts. While the female ids are fixed and obtained from the `data.fgender` file, the popular ids depends on `is_popular_alg` and is calculated based on the data distribution

- `adila/{popularity,gender}/{notion}`, the subfolder containg the results based on the fairness notion
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred`, the reranked version of the recommended members from `data.fpred`
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred.eval.fair.instance.csv`, the fairness metric values for `data.fpred` (before) and the reranked version (after) per each team instance in the test set  
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred.eval.fair.mean.csv`, the average of fairness metric values for `data.fpred` (before) and the reranked version (after) over all teams of the test set  
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred.eval.utility.instance.csv`, the accuracy metric values for `data.fpred` (before) and the reranked version (after) per each team instance in the test set  
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred.eval.utility.mean.csv`, the average of accuracy metric values for `data.fpred` (before) and the reranked version (after) over all teams of the test set

In [10]:
!find ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/ -print | sed 's;[^/]*/;│   ;g;s;│   \([^│]\);├── \1;'

│   │   │   │   │   │   │   
│   │   │   │   │   │   ├── popularity
│   │   │   │   │   │   │   ├── labels.csv
│   │   │   │   │   │   │   ├── dp
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_const_sort.auc.5.rerank.pred
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_const_sort.avg.5.rerank.pred
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_relaxed.avg.5.rerank.pred.eval.fair.mean.csv
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_const_sort.auc.5.rerank.pred.eval.fair.mean.csv
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_const_sort.avg.5.rerank.pred.eval.fair.mean.csv
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_greedy.avg.5.rerank.pred
│   │   │   │   │   │   │   │   ├── f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.fair.instance.csv
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_greedy.avg.5.rerank.pred.eval.fair.instance.csv
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_cons.auc.5.rerank.pred.eval.fair.mean.csv
│   │   │   │   │

**Average fairness measures**






In [11]:
import pandas as pd
pd.read_csv('../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/popularity/eo/f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.fair.mean.csv', index_col=0)

,mean
metrics,
before.ndkl,0.289672
after.ndkl,0.140630
before.skew.minority,-9.715486
before.skew.majority,-0.028526
after.skew.minority,-9.715486
after.skew.majority,-0.028526


**Fairness measures per each test team**

In [12]:
import pandas as pd
pd.read_csv('../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/popularity/eo/f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.fair.instance.csv')

,before.ndkl,after.ndkl,before.skew.minority,before.skew.majority,after.skew.minority,after.skew.majority
0,0.105361,0.105361,-25.328436,0.105361,-25.328436,0.105361
1,0.105361,0.105361,-25.328436,0.105361,-25.328436,0.105361
2,0.095459,0.095459,0.693147,-0.117783,0.693147,-0.117783
3,0.943696,0.198485,0.693147,-0.117783,0.693147,-0.117783
4,0.198485,0.198485,0.693147,-0.117783,0.693147,-0.117783


**Average accuracy values (Utility)**


In [13]:
import pandas as pd
pd.read_csv('../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/popularity/eo/f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.utility.mean.csv', index_col = 0)

,mean.before,mean.after
metric,,
P_1,0.200000,0.200000
P_2,0.200000,0.200000
P_5,0.120000,0.120000
recall_1,0.100000,0.100000
recall_2,0.200000,0.200000
recall_5,0.300000,0.300000
ndcg_cut_1,0.200000,0.200000
ndcg_cut_2,0.200000,0.200000
ndcg_cut_5,0.252814,0.252814


**Accuracy values per each test team (Utility)**


In [14]:
import pandas as pd
pd.read_csv('../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/popularity/eo/f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.utility.instance.csv')

,P_1.before,P_2.before,P_5.before,recall_1.before,recall_2.before,recall_5.before,ndcg_cut_1.before,ndcg_cut_2.before,ndcg_cut_5.before,map_cut_1.before,...,recall_5.after,ndcg_cut_1.after,ndcg_cut_2.after,ndcg_cut_5.after,map_cut_1.after,map_cut_2.after,map_cut_5.after,success_1.after,success_2.after,success_5.after
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.00,0.000,0.0,0.0,0.0
1,1.0,0.5,0.2,0.5,0.5,0.5,1.0,0.61315,0.61315,0.5,...,0.5,1.0,0.61315,0.61315,0.5,0.50,0.500,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.00,0.000,0.0,0.0,0.0
3,0.0,0.0,0.2,0.0,0.0,0.5,0.0,0.00000,0.26407,0.0,...,0.5,0.0,0.00000,0.26407,0.0,0.00,0.125,0.0,0.0,1.0
4,0.0,0.5,0.2,0.0,0.5,0.5,0.0,0.38685,0.38685,0.0,...,0.5,0.0,0.38685,0.38685,0.0,0.25,0.250,0.0,1.0,1.0


**2. Running Adila (Bulk)**

To run on multiple prediction files in `parallel`, common for cross-fold validations of recommender models, `data.fpred` can be a folder cotaining the `*.pred` files related to the same dataset like:

```
acceleration='cpu' # all cores but one, or 'cpu:4' for four cores only
data.fpred=../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/
```



In [15]:
!python main.py data.fpred=../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/ fair.algorithm=fa-ir fair.notion=eo fair.attribute=popularity fair.is_popular_alg=avg

[2025-12-09 05:54:45,121][__main__][INFO] - Loading stats, ratios, and ids for minority experts ...
main.py:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  preds = torch.l

**3. Running Adila (Bruteforce)**

To run for all different fairness settings on single or multiple prediction files, common for experiments, [`./src/grid.py`](./src/grid.py) to perform:

```
for fair.algorithm in ['fa-ir', 'det_greedy', 'det_relaxed', 'det_const_sort', 'det_cons']:
    for fair.notion in ['eo', 'dp']:
        for fair.attribute in ['popularity', 'gender']:
            for fair.is_popular_alg in ['avg', 'auc']:

```

In [17]:
!python grid.py data.fpred=../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/ acceleration=cpu:1

[2025-12-09 05:56:03,657][main][INFO] - Loading stats, ratios, and ids for minority experts ...
/content/adila/src/main.py:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  